## threading 的多线程并发
- 对比多进程并发： 
  - 优点 ： 资源消耗少
  - 缺点 ： 需要注意共享该资源的争夺  (注意GIL问题)
- 实现步骤
  1. 创建套接字，绑定，监听
  2. 接收客户端请求，创建新的线程
  3. 主线程继续等待其他客户端连接，分支线程执行客户    端请求
  4. 处理完客户端请求后，分支线程退出，关闭客户端套    接字

In [ ]:
# thread_server
from socket import * 
from threading import * 
import sys

HOST = '0.0.0.0'
PORT = 8888
ADDR = (HOST,PORT)

#创建套接字
s = socket()
s.bind(ADDR)
s.listen(5)

#客户端处理函数
def handler(connfd):
    print("Got connection from",connfd.getpeername())
    while True:
        data = connfd.recv(1024).decode()
        if not data:
            break
        connfd.send(b"receive your message")
    connfd.close()


while True:
    try:
        connfd,addr = s.accept()
    except KeyboardInterrupt:
        s.close()
        sys.exit("服务器退出")
    except Exception as e:
        print(e)
        continue 

    t = Thread(target = handler,args = (connfd,))
    t.setDaemon(True)
    t.start()


## socket 服务器继承模块
- python2 SocketServer
- python3 socketserver

- 功能:通过模块的接口完成基于多进程/多线程的tcp/udp 的socket并发程序

```python

#模块类
import socketserver
dir(socketserver)

DatagramRequestHandler  处理udp请求 
StreamRequestHandler    处理tcp请求

UDPServer   提供udp服务端类   
TCPServer   提供tcp服务端类

ForkingMixIn   提供进程创建
ForkingTCPServer   ==》ForkingMixIn + TCPServer
ForkingUDPServer   ==》ForkingMixIn + UDPServer

ThreadingMixIn  提供线程创建
ThreadingTCPServer  ==》ThreadingMixIn + TCPServer
ThreadingUDPServer  ==》ThreadingMixIn + TCPServer

```

In [2]:
from socketserver import * 

# 创建多进程  tcp 并发

# 创建进程tcp服务器类
# class Server(ForkingTCPServer):
# class Server(ForkingMixIn,TCPServer):
    # pass 

#多线程tcp并发
class Server(ThreadingTCPServer):
    pass

#具体请求处理类
class Handler(StreamRequestHandler):
    def handle(self):
        #self.request 相当于accept创建的套接字
        print("Connect from",self.request.getpeername())
        while True:
            data = self.request.recv(1024).decode()
            if not data:
                break
            print(data)
            self.request.send(b"Receive your message")

#生产服务器对象,传入addr和具体处理类
server = Server(("0.0.0.0",8888),Handler)

#启动服务器
server.serve_forever()

In [ ]:
from socketserver import * 

#多进程 udp并发
class Server(ForkingMixIn,UDPServer):
    pass 

class Handler(DatagramRequestHandler):
    def handle(self):
        while True:
            #接收消息
            data = self.rfile.readline().decode()
            if not data:
                break
            print(data)
            #发送消息
            self.wfile.write\
            (b"Receive your message")

server = Server(('127.0.0.1',8888),Handler)
server.serve_forever()